## Predatory Inclusion in Non-Profit and For-Profit Online Education
### By Christian Michael Smith, Amber D. Villalobos (equal authors), Laura T. Hamilton, and Charlie Eaton

### Code by Christian Smith

### Estimation for Tables 3 and 5

In [1]:
*ssc install univar

In [2]:
use data/d_t3_t5_ipeds_scorecard, clear

In [3]:
describe, size


Contains data from data/d_t3_t5_ipeds_scorecard.dta
 Observations:        67,461                  
    Variables:           968                  21 Feb 2022 21:41
         Size:   581,513,820                  
--------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
--------------------------------------------------------------------------------
unitid          long    %12.0g                UNITID
instnm          str105  %105s                 INSTNM
addr            str81   %81s                  ADDR
city            str28   %28s                  CITY
stabbr          str2    %9s                   STABBR
zip             str10   %10s                  ZIP
fips            byte    %8.0g                 FIPS
obereg          byte    %8.0g                 OBEREG
chfnm           str50   %50s                  CHFNM
chftitle        str75   %75s                  CHFTITLE
gen

In [4]:
set more off

In [5]:
******************************************
***************(2) ANALYSIS***************
******************************************

**********(A) SETUP DATA FOR ANALYSIS**********

*Keep only 4yr institutions
keep if iclevel==1

*Drop observations from the 2020-2021 academic year 
drop if year==2020

*Define sets of control variables
	*Control variable set when dependent variable is student body demographic in nature
global controls_demog "i.year i.control i.hdegofr1 i.locale i.obereg hbcu"
	*Control variable set when dependent variable is a student outcome
global controls_outcome "i.year logenrl blackpct_u latinxpct_u aianpct_u asiapct_u malepct_u upgrntp acceptrate i.control i.hdegofr1 i.locale i.obereg hbcu"


**********(B) ESTIMATE %ONLINE --> DEPENDENT VARIABLE RELATIONSHIPS**********

*Estimate %online --> student demographics relationships (unconditional)
foreach demogvar in upgrntp blackpct_u {
	foreach control in allcontrols forprofit nonprofit {
		reg `demogvar' pctonline10 if `control'==1, vce(cluster unitid)
		display "Dependent variable:  `demogvar',  Control: `control'"
	}
}


*Estimate %online --> student demographics relationships (conditional on baseline institutional characteristics)
foreach demogvar in upgrntp blackpct_u {
	foreach control in allcontrols forprofit nonprofit {
		reg `demogvar' pctonline10 $controls_demog if `control'==1, vce(cluster unitid)
		display "Dependent variable:  `demogvar',  Control: `control'"
	}
}

*Estimate %online --> student outcome relationships
foreach outcomevar in ret_pcf gradrate6 totloan_a bbrr2_fed_ug_dflt bbrr2_fed_ug_dlnq bbrr2_fed_ug_fbr bbrr2_fed_ug_noprog bbrr2_fed_ug_discharge bbrr2_fed_ug_dfr  bbrr2_fed_ug_makeprog bbrr2_fed_ug_paidinfull {
	foreach control in allcontrols forprofit nonprofit {
		reg `outcomevar' pctonline10 $controls_outcome if `control'==1, vce(cluster unitid)
		display "Dependent variable:  `outcomevar',  Control: `control'"
	}
}


(39,191 observations deleted)

(2,898 observations deleted)




Linear regression                               Number of obs     =     15,994
                                                F(1, 2552)        =      32.62
                                                Prob > F          =     0.0000
                                                R-squared         =     0.0177
                                                Root MSE          =      18.51

                             (Std. err. adjusted for 2,553 clusters in unitid)
------------------------------------------------------------------------------
             |               Robust
     upgrntp | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
 pctonline10 |   1.034454   .1811158     5.71   0.000     .6793052    1.389603
       _cons |   41.07298   .4233985    97.01   0.000     40.24274    41.90322
-----------------------------

In [7]:
**********(C) COMPUTE DESCRIPTIVE STATISTICS**********
set line 255
*Describe continuous variables
univar pctonline blackpct_u ret_pcf gradrate6 totloan_a bbrr2_fed_ug_dflt bbrr2_fed_ug_dlnq bbrr2_fed_ug_fbr bbrr2_fed_ug_dfr bbrr2_fed_ug_noprog bbrr2_fed_ug_makeprog bbrr2_fed_ug_paidinfull bbrr2_fed_ug_discharge logenrl latinxpct_u aianpct_u asiapct_u malepct_u upgrntp acceptrate hbcu

*Describe categorical variables
tab year 
tab control 
tab hdegofr1 
tab locale 
tab obereg

*Show online shares by sector and year 
foreach sector in nonprofit forprofit {
	forvalues yr=2012/2019 {
		summ pctonline if year==`yr' & `sector'==1
		display "Sector:  `sector',  Year: `yr'"
	}
}




                                        -------------- Quantiles --------------
Variable       n     Mean     S.D.      Min      .25      Mdn      .75      Max
-------------------------------------------------------------------------------
pctonline   16423    14.79    24.36     0.00     0.24     3.91    16.37   100.00
blackpct_u   21560    15.38    19.97     0.00     3.54     7.82    18.32   100.00
 ret_pcf   17399    71.75    18.39     0.00    64.00    75.00    83.00   100.00
gradrate6    6185    52.88    20.50     1.79    38.24    52.79    66.86   100.00
totloan_a   14049 17401.62  5434.57   227.00 13823.00 16972.00 20308.00 57428.00
bbrr2_fed_ug_dflt    1663     7.83     4.67     0.93     4.45     6.65    10.26    31.74
bbrr2_fed_ug_dlnq    1253     4.82     2.25     0.78     3.15     4.29     5.88    14.62
bbrr2_fed_ug_fbr    2077    17.26     8.46     2.64    10.94    15.67    22.69    46.33
bbrr2_fed_ug_dfr    2065    13.71     3.38     2.70    11.85    13.65    15.47    48.78